In [2]:
# Libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 2000)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Remove printing error
pd.options.mode.chained_assignment = None

In [3]:
# Import
path = r'switch_data/c22_all_with_diff_2023.csv'
c22_df = pd.read_csv(path)

In [4]:
# Import
path = r'switch_data/antibiotic_po_flag_2023.csv'
antibiotic_df = pd.read_csv(path)

In [11]:
# Conver to datetime 
c22_df['date'] =  pd.to_datetime(c22_df['date'])
antibiotic_df['ADMINISTRATION_DATETIME'] =  pd.to_datetime(antibiotic_df['ADMINISTRATION_DATETIME'])

In [ ]:
### Need to drop those where days are not continous as above!!!

In [19]:
drop_stays = []
for spell in c22_df.SPELL_IDENTIFIER.unique():
    temp_df = c22_df[c22_df['SPELL_IDENTIFIER'] == spell]
    temp_df.reset_index(inplace=True, drop=True)
    for index, row in temp_df.iterrows():
        #print(row)
        if index == 0:
            #print('first')
            continue
        elif row.date == temp_df.iloc[index-1]['date'] + pd.DateOffset(1):
            #print(row)
            #print(temp_df)
            continue
        else:
            drop_stays.append(spell)

c22_df2 = c22_df[~c22_df['SPELL_IDENTIFIER'].isin(drop_stays)]

In [23]:
c22_df2.SPELL_IDENTIFIER.nunique()
antibiotic_df.SPELL_IDENTIFIER.nunique()
len(list(set(c22_df2.SPELL_IDENTIFIER.unique()) & set(antibiotic_df.SPELL_IDENTIFIER.unique())))

2244

3431

2244

In [25]:
# Rename
antibiotic_df.rename(columns={'ADMINISTRATION_DATETIME': 'date'}, inplace=True)

In [28]:
# Merge features and labels
icare_df = pd.merge(antibiotic_df[['SPELL_IDENTIFIER', 'date', 'ROUTE', 'po_flag', 'iv_treatment_length']], c22_df2)

In [30]:
# Order
icare_df.sort_values(by=['SPELL_IDENTIFIER', 'date'], inplace=True)

In [31]:
# Reset index 
icare_df.reset_index(inplace=True, drop=True)

In [33]:
# Save
#icare_df.to_csv('switch_data/icare_switch_data_2023.csv', index=False)

In [ ]:
# Import
#path = r'switch_data/icare_switch_data.csv'
#icare_df = pd.read_csv(path)

In [35]:
# Function to remove outliers based using Q(1/3) -+ 2 * IQR
def remove_data_outliers(df, coeff=6, tol=0):
    """Utility function to remove data outliers from the data set using Z-Score method.
    Args:
        df (DataFrame): Input transformed data set.
        coeff (float, optional): Scaling coefficient for outlier removal strategy. Defaults to 6.
        tol (int, optional): Feature tolerance on how many features with NaN values may exist. Defaults to 0.
    Returns:
        Data set (DataFrame): Output data set without outlier values.
        Outliers per variable (Series): Number of outliers remaining per variable.
    """

    # Calculate IQR = Q3 - Q1
    q1, q3 = df.quantile(.25), df.quantile(.75)
    IQR = q3 - q1
    lower_bound = q1 - (coeff * IQR)
    upper_bound = q3 + (coeff * IQR)

    # Set the outliers to unique number
    df[(df < lower_bound) | (df > upper_bound)] = 999999999

    # Store number of outliers per column
    outlier_count = df[df == 999999999].count(axis=0)

    # Set the outliers to NaN
    df = df.replace(999999999, np.NaN)
    #outlier_count2 = df[df == 999999999].count(axis=0)

    # Drop NaN values with option for adjusting threshold
    #df = df.dropna(how='any', thresh=df.shape[1] - tol)

    # Store number of outliers per column
    #outlier_count = df.isna().sum(axis=0)

    # Return complete profiles and outliers per column
    return df, outlier_count#, outlier_count2

In [36]:
from sklearn import preprocessing

def preprocess_fun(data):
    # Split X y
    X = data.drop(columns=['SPELL_IDENTIFIER', 'date', 'ROUTE', 'po_flag', 'iv_treatment_length', '24_hour_flag', '48_hour_flag'])
    y = data['po_flag']
    # Remove outliers
    times = 2
    X2, outlier_count = remove_data_outliers(X, coeff=times)
    # Forward fill based on stay_id and -1 fill missing
    X2 = pd.concat([X2, data['SPELL_IDENTIFIER']], axis=1)
    ffilled_data = X2.groupby('SPELL_IDENTIFIER').transform(lambda p: p.ffill())
    #ffilled_data = ffilled_data.join(combined_data['stay_id'])
    ffilled_features = ffilled_data
    # Filling missing data, normalize features
    min_max_scaler = preprocessing.MinMaxScaler()
    data_scaled = min_max_scaler.fit_transform(ffilled_features)
    processed_data = np.nan_to_num(data_scaled, nan=-1)

    # Add back lables
    feature_names = list(ffilled_features.columns)
    X3 = pd.DataFrame(processed_data, columns=feature_names)
    #X3.drop(columns=['stay_id'], inplace=True)

    # Split into training, validation and testing
    model_data = pd.concat([data[['SPELL_IDENTIFIER', 'date', 'ROUTE', 'po_flag', 'iv_treatment_length', '24_hour_flag', '48_hour_flag']], X3], axis=1)

    return model_data

In [37]:
# Preprocess
icare_df_preprocessed = preprocess_fun(icare_df)

All-NaN slice encountered
All-NaN slice encountered


In [42]:
# Save
#icare_df_preprocessed.to_csv('switch_data/icare_switch_data_preprocessed_2023.csv', index=False)